In [1]:
import numpy as np
import pandas as pd
from datetime import date, datetime
from data_preparation import extract_ohlcv_from_binance_data, concatenate_dataframes
from finta import TA
from sklearn.model_selection import train_test_split

In [2]:
pairs = ['ADA_USDT', 'BTC_USDT', 'DOGE_USDT', 'DOT_USDT', 'ETH_USDT', 'LINK_USDT', 'SOL_USDT', 'TRX_USDT', 'XRP_USDT']
pair_data = []
for pair in pairs:
    df = pd.read_feather(f'../ft_userdata/user_data/data/binance/{pair}-1m.feather')
    df, test_df = train_test_split(df, test_size=0.15, shuffle=False) # не хотим подглядеть в будущее
    pair_data.append(df)

In [3]:
start_max = pair_data[0]['date'].iloc[0]
end_min = pair_data[0]['date'].iloc[-1]
for df in pair_data:
    start_max = max(df['date'].iloc[0], start_max)
    end_min = min(df['date'].iloc[-1], end_min)
print(f"start: {start_max}, end: {end_min}")

start: 2020-08-18 23:00:00+00:00, end: 2024-03-30 16:53:00+00:00


In [4]:
def generate_target_classification(df, fee = 0.001):
    df['is_big_increase'] = (df['close'].shift(-1) / df['close']) > 1 + 2 * fee
    df['is_drawdown'] = (df['close'].shift(-1) / df['close']) < 1
    return df

In [5]:
num_bps = 10
fee = num_bps * 0.0001 # 10 процентных пунктов
print(f'Доля изменения цены на >= {num_bps} п.п. вверх для каждой пары')
for df, pair in zip(pair_data, pairs):
    df = generate_target_classification(df, fee)
    big_increase_ratio = 100 * df['is_big_increase'].sum() / len(df)
    print(f"Пара: {pair:<9} - {big_increase_ratio:.2f}%")

Доля изменения цены на >= 10 п.п. вверх для каждой пары
Пара: ADA_USDT  - 5.82%
Пара: BTC_USDT  - 2.05%
Пара: DOGE_USDT - 6.53%
Пара: DOT_USDT  - 6.33%
Пара: ETH_USDT  - 3.42%
Пара: LINK_USDT - 7.53%
Пара: SOL_USDT  - 8.45%
Пара: TRX_USDT  - 4.10%
Пара: XRP_USDT  - 4.63%
